In [1]:
# Reset variables
%reset -f

In [2]:
# Import necessary libraries
import os
import json
import copy
import math
import datetime
import numpy as np
import pandas as pd
from scipy import interpolate
import scipy.stats as st

# smoothing models
from statsmodels.nonparametric.smoothers_lowess import lowess
from scipy.signal import savgol_filter

In [3]:
name = "Full_Model_ywarptest"

In [13]:
# Define some helper functions
def create_workspace_folders():
    date = datetime.datetime.now()
    workspace = "workspace\{}-{}".format(date.strftime("%Y_%m_%d"), name)

    create_folders = ['workspace', workspace]
    # create workspace
    for folder in create_folders:
        if not os.path.exists(folder):
            os.makedirs(folder)
    
    return workspace

def smoothing(a, window_size, method="moving"):
    if method == "moving":
        return moving_average(a, window_size)
    elif method == "lowess":
        return lowess_smoothing(a, window_size)
    elif method == "sgolay":
        return sgolay_smoothing(a, window_size)

def sgolay_smoothing(a, window_size):
    if window_size % 2 == 0:
        window_size -= 1
    
    # clip the array if there are preceeding 0s
    non_zero_idx = 0
    for idx in range(a.shape[0]):
        if a[idx] != 0:
            non_zero_idx = idx
            break
    
    # create cropped array
    sgolay_smoothed = copy.deepcopy(a)    
    a_cropped = a[non_zero_idx:] # from index to 0
    cropped_smoothed = savgol_filter(a_cropped, window_size, 2)
    sgolay_smoothed[non_zero_idx:] = cropped_smoothed
    
    return sgolay_smoothed
    
def lowess_smoothing(a, window_size):
    # clip the array if there are preceeding 0s
    non_zero_idx = 0
    for idx in range(a.shape[0]):
        if a[idx] != 0:
            non_zero_idx = idx
            break
    
    # create cropped array
    lowess_smoothed = copy.deepcopy(a)
    a_cropped = a[non_zero_idx:] # from index to 0
    cropped_smoothed = lowess(a_cropped, np.arange(a_cropped.shape[0]), 
                  frac=window_size/a_cropped.shape[0], it=0)[:, 1]
    lowess_smoothed[non_zero_idx:] = cropped_smoothed

    return lowess_smoothed 
    
def moving_average(a, window_size):
    new_a = []
    half_window = round(window_size/2)
    for idx, val in enumerate(a):
        if val == 0:
            new_a.append(0)
            continue
            
        start_idx = max(idx-half_window, 0)
        end_idx = min(idx+half_window+1, a.shape[0])
        a_segment = a[start_idx:end_idx]
        
        a_seg_mean = np.nanmean(np.where(a_segment!=0, a_segment, np.nan)) # nonzero mean
        new_a.append(a_seg_mean)
        
    return new_a

def hampel_filter(input_series, window=5, n_sigmas=1):
    # ensure the data is flattened
    input_series = np.array(input_series).flatten()
    
    # returns filtered timepoints and coordinates along
    n = len(input_series)
    k = 1.4826 # scale factor for Gaussian distribution
    
    outliers_idxs = []
    outliers_filtered = []
    extended_series = np.pad(input_series, (window, window), 'reflect')
    filtered_series = extended_series.copy()
    n_ex = len(extended_series)
    for i in range(window_size, n_ex - window_size):
        x0 = np.median(
            extended_series[(i - window_size):(i + window_size)])
        S0 = k * np.median(np.abs(
            extended_series[(i - window_size):(i + window_size)] - x0))
        if (np.abs(extended_series[i] - x0) > n_sigmas * S0):
            filtered_series[i] = x0 # replaces outlier with median
            outliers_idxs.append(i-window) # logs outlier index
            outliers_filtered.append(x0) # the value that replaces outlier
    
    filtered_series = filtered_series[window:len(filtered_series)-window]
    
    return filtered_series, outliers_idxs, outliers_filtered

def create_error_dict(strain="", pos="", volumes=[], annotations=[], cell_names=[], error_type="", notes=""):
    error_dict = {
        'strain': strain,
        'position': pos,
        'volumes': ','.join(np.array(volumes).astype(str).tolist()),
        'annotations': ','.join(np.array(annotations).astype(str).tolist()),
        'cell_names': ','.join(np.array(cell_names).astype(str).tolist()),
        'error_type': error_type,
        'notes': notes
    }
    return error_dict

def create_error_xlsx(workspace_folderpath, errors):
    if not errors:
        return
    
    error_path = os.path.join(workspace_folderpath, 'errors.xlsx')
    xlsx_writer = pd.ExcelWriter(error_path, engine='xlsxwriter')
    
    errors_pd = pd.DataFrame(errors)
    
    # find unique strains
    strains = errors_pd['strain'].unique()
    
    for strain in strains:
        # find the appropriate errors corresponding to strain
        strain_errors = errors_pd.loc[errors_pd['strain'] == strain]
        # save to excel sheet
        strain_errors.to_excel(xlsx_writer, sheet_name=strain)
    
    # save file
    xlsx_writer.save()

def write_error_txt(workspace_folderpath, error, write_type="w"):    
    error_path = os.path.join(workspace_folderpath, 'errors.txt')
    
    with open(error_path, write_type) as f:
        f.write(error)

# try thin plate spline warping
def thin_plate_spline_warp(unwarped_pts, ctrl_pts, obj_to_warp):

    # convert everything to np array
    unwarped_pts = np.array(unwarped_pts)
    ctrl_pts = np.array(ctrl_pts)
    obj_to_warp = np.array(obj_to_warp)

    num_points = unwarped_pts.shape[0]
    K = np.zeros((num_points, num_points))
    for rr in np.arange(num_points):
        for cc in np.arange(num_points):
            K[rr,cc] = np.sum(np.subtract(unwarped_pts[rr,:], unwarped_pts[cc,:])**2) #R**2 
            K[cc,rr] = K[rr,cc]

    #calculate kernel function R
    K = np.maximum(K, 1e-320) 
    #K = K.* log(sqrt(K))
    K = np.sqrt(K) #
    # Calculate P matrix
    P = np.hstack((np.ones((num_points, 1)), unwarped_pts)) #nX4 for 3D
    # Calculate L matrix
    L_top = np.hstack((K, P))
    L_bot = np.hstack((P.T, np.zeros((4,4))))
    L = np.vstack((L_top, L_bot))

    param = np.matmul(np.linalg.pinv(L), np.vstack((ctrl_pts, np.zeros((4,3)))))
    # Calculate new coordinates (x',y',z') for each points 
    num_points_obj = obj_to_warp.shape[0]

    K = np.zeros((num_points_obj, num_points))
    gx = obj_to_warp[:,0]
    gy = obj_to_warp[:,1]
    gz = obj_to_warp[:,2]

    for nn in np.arange(num_points):
        K[:,nn] = \
        np.square(np.subtract(gx, unwarped_pts[nn,0])) + \
        np.square(np.subtract(gy, unwarped_pts[nn,1])) + \
        np.square(np.subtract(gz, unwarped_pts[nn,2])) # R**2
 
    K = np.maximum(K, 1e-320) 
    K = np.sqrt(K) #|R| for 3D
    gx = np.vstack(obj_to_warp[:,0])
    gy = np.vstack(obj_to_warp[:,1])
    gz = np.vstack(obj_to_warp[:,2])
    P = np.hstack((np.ones((num_points_obj,1)), gx, gy, gz))
    L = np.hstack((K, P))
    object_warped = np.matmul(L, param)
    object_warped[:,0] = np.round(object_warped[:,0]*10**3)*10**-3
    object_warped[:,1] = np.round(object_warped[:,1]*10**3)*10**-3
    object_warped[:,2] = np.round(object_warped[:,2]*10**3)*10**-3

    return object_warped

def thin_plate_spline_warp_c_elegans(unwarped_pts, ctrl_pts, obj_to_warp):
    """Warps, then shifts xy based on bounding box. The goal is to have everything on the xz plane/y=0
    before we start warping.
    """
    # bring everything up to the unwarped plane in x and y. z is length-wise
    unwarped_half_bound = np.mean(unwarped_pts[:, 1])
    x_shift = unwarped_half_bound - (np.mean(ctrl_pts[:, 0]))
    y_shift = unwarped_half_bound - (np.mean(ctrl_pts[:, 1]))
    
    ctrl_pts_shifted = copy.deepcopy(ctrl_pts)
    ctrl_pts_shifted[:, 0] = ctrl_pts[:, 0] + x_shift
    ctrl_pts_shifted[:, 1] = ctrl_pts[:, 1] + y_shift
    
    # perform warp
    object_warped = thin_plate_spline_warp(unwarped_pts, ctrl_pts_shifted, obj_to_warp)
    """
    ctrl_pts[:, 0] += x_shift
    ctrl_pts[:, 1] += y_shift
    
    # perform warp
    object_warped = thin_plate_spline_warp(unwarped_pts, ctrl_pts, obj_to_warp)
    """

    # subtract bounding box in x and y
    object_warped[:, 0:2] -= unwarped_half_bound # match for points to warp. seam cells and annotations
    
    for coord_idx, coord in enumerate(obj_to_warp):
        # if at the origin, then keep y at origin too
        if coord[0] == 0 and coord[2] == 0:
            object_warped[coord_idx, 1] = 0
            
    return object_warped

def y_warp_correction(unwarped_pts, ctrl_pts, old_coords, new_coords):
    # calculate average x and y
    # bring everything up to the unwarped plane in x and y. z is length-wise
    unwarped_avg = [np.mean(unwarped_pts[:, 0]), np.mean(unwarped_pts[:, 1])]
    ctrl_pt_avg = [np.mean(ctrl_pts[:, 0]), np.mean(ctrl_pts[:, 1])]
    
    # print(unwarped_avg, ctrl_pt_avg)
    
    # centered at x=0, y=0
    old_coords_centered = old_coords.copy()
    # print('old pre', old_coords_centered)
    old_coords_centered[:, 0] -= unwarped_avg[0]
    old_coords_centered[:, 1] -= unwarped_avg[1]
    # print('old translated', old_coords_centered)

    new_coords_centered = new_coords.copy()
    """
    print('new pre', new_coords_centered)
    new_coords_centered[:, 0] -= ctrl_pt_avg[0]
    new_coords_centered[:, 1] -= ctrl_pt_avg[1]
    print('new translated', new_coords_centered)
    """
    
    # calculate ratios
    old_coords_centered_np = np.array(old_coords_centered[:, 0]).astype(float)
    x_old_no_zero = np.where(old_coords_centered_np==0, 1e-15, old_coords_centered_np) 
    y_x_old = np.divide(np.array(old_coords_centered[:, 1]).astype(float), x_old_no_zero)
    y_new = np.multiply(y_x_old, new_coords_centered[:, 0])

    # print('y_x_old', y_x_old)
    # print('x_old', new_coords_centered[:, 0])
    # print('y_old', new_coords_centered[:, 1])
    # print('y_new', y_new)
    
    # calculate change in y
    # y_diff = y_new - new_coords_centered[:, 1]
    
    # apply to new coords
    new_coords[:, 1] = y_new
    
    return new_coords

In [5]:
# Setup
# Create name + workspace folders
workspace_folderpath = create_workspace_folders()
print('Workspace folderpath: {}'.format(workspace_folderpath))

# Load necessary file structures via config file
with open('config.json') as f:
    config = json.load(f)

Workspace folderpath: workspace\2021_01_08-Full_Model_ywarptest


In [6]:
# STEP 1: Import all the necessary data
def parse_mipav_data(config, cell_key, strain, pos_folderpath):
    # define output variables
    all_seam_cells = set(config['settings']['interpolation']['seam_cells_on'].keys())
    lattice_seam_cells = set(config['settings']['warping']['seam_cells'])
    seam_cell_output = {}
    annotation_output = {} # structured cell_name: pandas dataframe (timepoint, x, y, z)
    errors = [] # note potential errors in volumes
    
    # determine default or options specific to strain
    folderpaths = config['settings']['folderpaths']
    for folder_data in folderpaths.keys():
        try:
            folderpaths[folder_data] = cell_key['folderpaths'][folder_data]
            print('Using specific {} folder structure'.format(folder_data))
        except:
            pass
    
    # find the numbers of the folders
    start_vol = int(cell_key['start'])
    end_vol = int(cell_key['end'])
    all_folderpaths = folderpaths.copy()
    for vol_idx, vol_num in enumerate(range(start_vol, end_vol+1)):
        # ignore outliers
        """
        if vol_num in cell_key['outliers']:
            continue"""
        
        # define the specific volume number folder
        vol_folder = all_folderpaths['data_folderpath'].replace('#', str(vol_num))
        vol_folderpath = os.path.join(
            pos_folderpath, all_folderpaths['side'], vol_folder)
        
        # check if the folder exists
        if not os.path.isdir(vol_folderpath):
            error_msg = "FOLDER DNE: {}".format(vol_folderpath)
            errors.append(create_error_dict(
                strain=strain['name'],
                pos=cell_key['name'],
                volumes=[vol_num],
                error_type="FOLDER DNE: Timepoint folder not found",
                notes=vol_folderpath
            ))
            print(error_msg)
            continue

        # print (debugging)
        # print("processing vol", vol_num)
        
        # define individual, full filepaths from drive
        all_filepaths_folderpaths = {}
        for full_folderpaths_key in all_folderpaths.keys():
            if full_folderpaths_key != "side" and \
                full_folderpaths_key != "data_folderpath":
                all_filepaths_folderpaths[full_folderpaths_key] = os.path.join(
                    vol_folderpath, all_folderpaths[full_folderpaths_key])
                # print(all_filepaths_folderpaths[full_folderpaths_key])

                # check to see if exists
                if not os.path.isfile(all_filepaths_folderpaths[full_folderpaths_key]):
                    error_msg = "FILE DNE: {}".format(
                        all_filepaths_folderpaths[full_folderpaths_key])
                    errors.append(create_error_dict(
                        strain=strain['name'],
                        pos=cell_key['name'],
                        volumes=[vol_num],
                        error_type="File DNE",
                        notes=all_filepaths_folderpaths[full_folderpaths_key]
                    ))
                    print(error_msg)
        
        ## get individual data by cell in pandas format
        try:
            # import twisted seam cells           
            tw_seam_cells_fp = all_filepaths_folderpaths['twisted_seam_cells']
            tw_seam_cells = pd.read_csv(tw_seam_cells_fp)            
            # import straighted seam cells
            st_seam_cells_fp = all_filepaths_folderpaths['straightened_seam_cells']
            st_seam_cells = pd.read_csv(st_seam_cells_fp)
        except:
            continue # file errors should already be logged.
            
        try:
            # import twisted annotations
            tw_annotations_fp = all_filepaths_folderpaths['twisted_annotations']
            tw_annotations = pd.read_csv(tw_annotations_fp)
            # import straightened annotations
            st_annotations_fp = all_filepaths_folderpaths['straightened_annotations']
            st_annotations = pd.read_csv(st_annotations_fp)
        except:
            tw_annotations = None
            st_annotations = None
            
        # ERROR CHECKING: Check for file content ----------------------
        if st_seam_cells.size == 0: # check if empty
            error_msg = "DATA ERROR: Empty seam cell file at {}".format(st_seam_cells_fp)
            errors.append(create_error_dict(
                strain=strain['name'],
                pos=cell_key['name'],
                volumes=[vol_num],
                error_type="Empty seam cell file",
                notes=st_seam_cells_fp
            ))
            print(error_msg)
        if st_annotations is not None and st_annotations.size == 0: # check if empty
            error_msg = "DATA ERROR: Empty annotation file at {}".format(st_annotations_fp)
            errors.append(create_error_dict(
                strain=strain['name'],
                pos=cell_key['name'],
                volumes=[vol_num],
                error_type="Empty annotation file",
                notes=st_annotations_fp
            ))
            print(error_msg)
            
        # use to check values of files
        data_file_list = [
            (tw_seam_cells, tw_seam_cells_fp), (tw_annotations, tw_annotations_fp),
            (st_seam_cells, st_seam_cells_fp), (st_annotations, st_annotations_fp)
        ]
        # check for if coords are actually numbers
        for data, fp in data_file_list:
            if data is None:
                continue
                
            coordinates = data[['x_voxels', 'y_voxels', 'z_voxels']]
            cell_ids = data['name'].values.tolist()
            for cell_idx, cell_coord in coordinates.iterrows():
                if not cell_coord.equals(cell_coord.astype(float)):
                    error_msg = "DATA ERROR: Non-float found for cell {} in {}".format(
                        str(cell_ids[cell_idx]), fp)
                    errors.append(create_error_dict(
                        strain=strain['name'],
                        pos=cell_key['name'],
                        volumes=[vol_num],
                        annotations=[cell_ids[cell_idx]],
                        error_type="Non-float value found",
                        notes=fp
                    ))
                    print(error_msg)
        
        # handle mismatch errors between twisted and untwisted spaces
        seam_cells_mismatch = list(set(st_seam_cells['name'].values.astype(str).tolist()) - 
                                   set(tw_seam_cells['name'].values.astype(str).tolist()))
        if st_annotations is not None:
            annotations_mismatch = list(set(st_annotations['name'].values.astype(str).tolist()) - 
                                       set(tw_annotations['name'].values.astype(str).tolist()))
        else:
            annotations_mismatch = []
            
        if seam_cells_mismatch:
            error_msg = "DATA ERROR: Mis-match between seam cells ({}) found in {} and {}".format(
                ", ".join(seam_cells_mismatch), st_seam_cells_fp, tw_seam_cells_fp)
            errors.append(create_error_dict(
                strain=strain['name'],
                pos=cell_key['name'],
                volumes=[vol_num],
                cell_names=seam_cells_mismatch,
                error_type="Mis-match between seam cells in twisted/untwisted",
                notes="{}, {}".format(st_seam_cells_fp, tw_seam_cells_fp)
            ))
            print(error_msg)
            
        if annotations_mismatch:
            error_msg = "DATA ERROR: Mis-match between annotated cells ({}) found in {} and {}".format(
                ", ".join(annotations_mismatch), st_annotations_fp, tw_annotations_fp)
            errors.append(create_error_dict(
                strain=strain['name'],
                pos=cell_key['name'],
                volumes=[vol_num],
                cell_names=annotations_mismatch,
                error_type="Mis-match between annotated cells in twisted/untwisted",
                notes="{}, {}".format(st_annotations_fp, tw_annotations_fp)
            ))
            print(error_msg)
            
        # check if lattice is complete
        tw_missing_lattice_cells = list(lattice_seam_cells - set(tw_seam_cells['name'].values.tolist()))
        st_missing_lattice_cells = list(lattice_seam_cells - set(st_seam_cells['name'].values.tolist()))
        if tw_missing_lattice_cells: # check if missing parts of twisted lattice
            error_msg = "LATTICE ERROR: Missing twisted seam cells ({}) in {}".format(
                ", ".join(tw_missing_lattice_cells), tw_seam_cells_fp)
            errors.append(create_error_dict(
                strain=strain['name'],
                pos=cell_key['name'],
                volumes=[vol_num],
                cell_names=tw_missing_lattice_cells,
                error_type="Seam cells missing in twisted lattice",
                notes="{}".format(tw_seam_cells_fp)
            ))
            print(error_msg)
        if st_missing_lattice_cells: # check if missing parts of straighted lattice
            error_msg = "LATTICE ERROR: Missing straightened seam cells ({}) in {}".format(
                ", ".join(st_missing_lattice_cells), st_seam_cells_fp)
            errors.append(create_error_dict(
                strain=strain['name'],
                pos=cell_key['name'],
                volumes=[vol_num],
                cell_names=st_missing_lattice_cells,
                error_type="Seam cells missing in straightened lattice",
                notes="{}".format(st_seam_cells_fp)
            ))
            print(error_msg)
            
        # check if extra seam cells exist/mismatch with warping model
        extra_seam_cells = list(set(st_seam_cells['name'].values.tolist()) - all_seam_cells)
        if extra_seam_cells: # check if there are extra seam cells
            for extra_seam_cell in extra_seam_cells:
                st_seam_cells = st_seam_cells[st_seam_cells['name'] != extra_seam_cell]
            # st_seam_cells.to_csv(st_seam_cells_fp, index=False)
            """
            error_msg = "DATA WARNING: Ignoring extra seam cells ({}) found in {}".format(
                ", ".join(extra_seam_cells), st_seam_cells_fp)
            errors.append(create_error_dict(
                strain=strain['name'],
                pos=cell_key['name'],
                volumes=[vol_num],
                cell_names=seam_cells_mismatch,
                error_type="Data Error: Mis-match between annotated cells in twisted/untwisted",
                notes="{}, {}".format(st_seam_cells_fp, tw_seam_cells_fp)
            ))
            print(error_msg)"""
            
        # END FILE ERROR CHECK -----------------------------------------------------------

        # SEAM CELLS: combine into a single data structure and catch data errors
        # go through each row of the file and log the appropriate information
        for idx, seam_row in st_seam_cells.iterrows():
            seam_cell_name = seam_row['name'].upper() # seam cells are upper case? just stay consistent
            if seam_cell_name not in seam_cell_output.keys():
                seam_cell_output[seam_cell_name] = {}
                seam_cell_output[seam_cell_name]['timepoints'] = []
                seam_cell_output[seam_cell_name]['coordinates'] = []

            # append to appropriate seam cell
            if vol_num not in seam_cell_output[seam_cell_name]['timepoints']:
                seam_cell_coords = seam_row[['x_voxels','y_voxels','z_voxels']].values.flatten().tolist()
                seam_cell_output[seam_cell_name]['coordinates'].append(seam_cell_coords)
                seam_cell_output[seam_cell_name]['timepoints'].append(vol_num)

        # ANNOTATIONS: combine into a single data structure and catch data errors
        try:
            for cell_id in cell_key['mapping'].keys():
                cell_name = cell_key['mapping'][cell_id].lower()
                if cell_name not in annotation_output.keys():
                    annotation_output[cell_name] = {}
                    annotation_output[cell_name]['timepoints'] = []
                    annotation_output[cell_name]['coordinates'] = []

                # check if the cell name exists in volume
                # print(vol_folderpath, st_annotations['name'], cell_id)
                cell_row = st_annotations.loc[st_annotations['name'] == cell_id]
                if not cell_row.empty:
                    # try to catch a few errors
                    if cell_row.shape[0] > 1:
                        error_msg = "DATA ERROR: Ignoring identical cell IDs ({}) for cell {} in {}".format(
                            cell_id, cell_name, st_annotations_fp)
                        errors.append(create_error_dict(
                            strain=strain['name'],
                            pos=cell_key['name'],
                            volumes=[vol_num],
                            annotations=[cell_id],
                            cell_names=[cell_name],
                            error_type="Identical Cell IDs",
                            notes="{}".format(st_annotations_fp)
                        ))
                        print(error_msg)
                        continue
                    
                    # if there is nothing wrong, then proceed
                    if vol_num not in annotation_output[cell_name]['timepoints']:
                        cell_coords = cell_row[['x_voxels','y_voxels','z_voxels']].values.flatten().tolist()
                        annotation_output[cell_name]['coordinates'].append(cell_coords)
                        annotation_output[cell_name]['timepoints'].append(vol_num)
        except:
            error_msg = "DATA ERROR: Failure to read file {}".format(st_annotations_fp)
            errors.append(create_error_dict(
                strain=strain['name'],
                pos=cell_key['name'],
                volumes=[vol_num],
                error_type="Failed to read file",
                notes="{}".format(st_annotations_fp)
            ))
            print(error_msg)
                
    return seam_cell_output, annotation_output, errors

def convert_cell_key_csv2json(csv_filepath):
    cell_key_json = {}
    # load in csv file
    cell_key = pd.read_csv(csv_filepath, header=None, engine='python')
    
    ## get necessary information
    cell_key_json['name'] = str(cell_key.iloc[0,0])
    cell_key_json['start'] = int(cell_key.iloc[1,0])
    cell_key_json['end'] = int(cell_key.iloc[1,1])
    try:
        cell_key_json['outliers'] = cell_key.iloc[2].astype(int).values.tolist()
    except:
        cell_key_json['outliers'] = [] # if there are none
    
    # grab mapping
    cell_key_json['mapping'] = {}
    for row_idx in range(3, cell_key.shape[0]): # mapping starts row idx 3
        cell_id = str(cell_key.iloc[row_idx,0])
        cell_name = str(cell_key.iloc[row_idx,1])
        cell_key_json['mapping'][cell_id] = cell_name

    return cell_key_json
    
def get_cell_key(pos_folderpath):
    # return the cell key in dict/json format and convert any existing 
    # cell key csv into json if the json version doesn't exist.
    cell_key_filepath_csv = os.path.join(pos_folderpath, 'CellKey.csv')
    cell_key_filepath_json = os.path.join(pos_folderpath, 'cell_key.json')
    
    # if the json exists, use it
    if os.path.isfile(cell_key_filepath_json):
        with open(cell_key_filepath_json) as f:
            return json.load(f)
    elif os.path.isfile(cell_key_filepath_csv):
        cell_key_json = convert_cell_key_csv2json(cell_key_filepath_csv)
        cell_key_json_dump = json.dumps(cell_key_json, sort_keys=True, indent=4)
        with open(cell_key_filepath_json, "w") as f: 
            f.write(cell_key_json_dump)
        return cell_key_json
    else:
        cell_key_error_msg = 'Missing cell key: {}'.format(cell_key_filepath_json)
        print(cell_key_error_msg)
        return None
                
errors = [] # running list of errors through whole program        
strain_info = config['data']['strains']
compiled_data = {}
for strain in strain_info:
    if strain['include']:
        if strain['name'] not in compiled_data.keys():
            compiled_data[strain['name']] = {}
                
        for pos_folderpath in strain['folderpaths']:
            print('CURRENTLY LOADING: {}'.format(pos_folderpath))
            # find and load cell key
            cell_key = get_cell_key(pos_folderpath)

            if not cell_key:
                continue
                
            # get data
            seam_cells, annotations, strain_errors = parse_mipav_data(
                config, cell_key, strain, pos_folderpath)
            
            # combine data
            pos_name = cell_key['name']
            compiled_data[strain['name']][pos_name] = {}
            compiled_data[strain['name']][pos_name]['cell_key'] = cell_key
            compiled_data[strain['name']][pos_name]['seam_cells'] = seam_cells
            compiled_data[strain['name']][pos_name]['annotations'] = annotations
            
            # log the necessary errors
            errors.extend(strain_errors)

# save information as intermediate step in workspace
compiled_json = json.dumps(compiled_data, sort_keys=True, indent=4)
compiled_json_filepath = os.path.join(
    workspace_folderpath, '1_compiled_data.json')
with open(compiled_json_filepath, "w") as f: 
    f.write(compiled_json)

# generate error log
create_error_xlsx(workspace_folderpath, errors)
print('Errors Logged.')

CURRENTLY LOADING: Y:\RyanC\Cell Tracking Project\OD1599_NU\OD1599_MostRecent\120619_Pos2\Decon_reg
CURRENTLY LOADING: Y:\RyanC\Cell Tracking Project\OD1599_NU\OD1599_MostRecent\112719_Pos3\Decon_Reg
CURRENTLY LOADING: Y:\RyanC\Cell Tracking Project\OD1599_NU\OD1599_MostRecent\112619_Pos0\Decon_reg
CURRENTLY LOADING: Y:\RyanC\Cell Tracking Project\DCR6485_RPM1_NU\011419_Pos0\Decon_reg
CURRENTLY LOADING: Y:\RyanC\Cell Tracking Project\DCR6485_RPM1_NU\011419_Pos4\Decon_reg
CURRENTLY LOADING: Y:\RyanC\Cell Tracking Project\DCR6485_RPM1_NU\021020_Pos2\Decon_Reg
CURRENTLY LOADING: Y:\RyanC\Cell Tracking Project\JCC596_NU\091119_Pos3\Decon_registered
CURRENTLY LOADING: Y:\RyanC\Cell Tracking Project\JCC596_NU\091119_Pos2\Decon_registered
CURRENTLY LOADING: Y:\RyanC\Cell Tracking Project\JCC596_NU\082619_Pos3\Decon_registered
CURRENTLY LOADING: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos0


C:\Users\xusz\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


CURRENTLY LOADING: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos4
CURRENTLY LOADING: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos2
CURRENTLY LOADING: Y:\RyanC\Cell Tracking Project\RW10131\052918_Pos0
FILE DNE: Y:\RyanC\Cell Tracking Project\RW10131\052918_Pos0\RegB\Decon_reg_56\Decon_reg_56_results\straightened_seamcells\straightened_seamcells.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\RW10131\052918_Pos0\RegB\Decon_reg_56\Decon_reg_56_results\straightened_annotations\straightened_annotations.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\RW10131\052918_Pos0\RegB\Decon_reg_56\Decon_reg_56_results\straightened_lattice\straightened_lattice.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\RW10131\052918_Pos0\RegB\Decon_reg_56\Decon_reg_56_results\seam_cell_final\seam_cells.csv
CURRENTLY LOADING: Y:\RyanC\Cell Tracking Project\RW10131\052918_Pos1
CURRENTLY LOADING: Y:\RyanC\Cell Tracking Project\RW10896\Post Twitching\Pos1\Decon_registered
FILE DNE: Y:\RyanC\C

FILE DNE: Y:\RyanC\Cell Tracking Project\RW10896\Post Twitching\Pos4\Decon_registered\RegB\Decon_reg_132\Decon_reg_132_results\straightened_annotations\straightened_annotations.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\RW10896\Post Twitching\Pos4\Decon_registered\RegB\Decon_reg_132\Decon_reg_132_results\integrated_annotation\annotations.csv
DATA ERROR: Failure to read file Y:\RyanC\Cell Tracking Project\RW10896\Post Twitching\Pos4\Decon_registered\RegB\Decon_reg_131\Decon_reg_131_results\straightened_annotations\straightened_annotations.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\RW10896\Post Twitching\Pos4\Decon_registered\RegB\Decon_reg_133\Decon_reg_133_results\straightened_seamcells\straightened_seamcells.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\RW10896\Post Twitching\Pos4\Decon_registered\RegB\Decon_reg_133\Decon_reg_133_results\straightened_annotations\straightened_annotations.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\RW10896\Post Twitching\Pos4\Decon_registered\RegB\D

FILE DNE: Y:\RyanC\Cell Tracking Project\RW10598\Pos4-bgsub_95_iteration_5\RegB\Decon_reg_120\Decon_reg_120_results\integrated_annotation\annotations.csv
LATTICE ERROR: Missing twisted seam cells (TR, TL, V6L, V6R) in Y:\RyanC\Cell Tracking Project\RW10598\Pos4-bgsub_95_iteration_5\RegB\Decon_reg_120\Decon_reg_120_results\seam_cell_final\seam_cells.csv
LATTICE ERROR: Missing straightened seam cells (TR, TL, V6L, V6R) in Y:\RyanC\Cell Tracking Project\RW10598\Pos4-bgsub_95_iteration_5\RegB\Decon_reg_120\Decon_reg_120_results\straightened_seamcells\straightened_seamcells.csv
DATA ERROR: Failure to read file Y:\RyanC\Cell Tracking Project\RW10598\Pos4-bgsub_95_iteration_5\RegB\Decon_reg_119\Decon_reg_119_results\straightened_annotations\straightened_annotations.csv
CURRENTLY LOADING: Y:\RyanC\Cell Tracking Project\RW10598\598_Slitscan_6um_5min_Pos1
FILE DNE: Y:\RyanC\Cell Tracking Project\RW10598\598_Slitscan_6um_5min_Pos1\RegB\Decon_reg_21\Decon_reg_21_results\straightened_seamcells\stra

...

FILE DNE: Y:\RyanC\Cell Tracking Project\Original Untwisting Paper\DCR4221\063014_lattices_Javier - UTP 1\Decon_reg\Decon_reg_214\Decon_reg_214_results\straightened_annotations\straightened_annotations.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\Original Untwisting Paper\DCR4221\063014_lattices_Javier - UTP 1\Decon_reg\Decon_reg_214\Decon_reg_214_results\integrated_annotation\annotations.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\Original Untwisting Paper\DCR4221\063014_lattices_Javier - UTP 1\Decon_reg\Decon_reg_215\Decon_reg_215_results\straightened_annotations\straightened_annotations.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\Original Untwisting Paper\DCR4221\063014_lattices_Javier - UTP 1\Decon_reg\Decon_reg_215\Decon_reg_215_results\integrated_annotation\annotations.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\Original Untwisting Paper\DCR4221\063014_lattices_Javier - UTP 1\Decon_reg\Decon_reg_216\Decon_reg_216_results\straightened_annotations\straightened_annotations.csv
FI

FILE DNE: Y:\RyanC\Cell Tracking Project\Original Untwisting Paper\DCR4221\063014_lattices_Javier - UTP 1\Decon_reg\Decon_reg_235\Decon_reg_235_results\straightened_annotations\straightened_annotations.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\Original Untwisting Paper\DCR4221\063014_lattices_Javier - UTP 1\Decon_reg\Decon_reg_235\Decon_reg_235_results\integrated_annotation\annotations.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\Original Untwisting Paper\DCR4221\063014_lattices_Javier - UTP 1\Decon_reg\Decon_reg_236\Decon_reg_236_results\straightened_annotations\straightened_annotations.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\Original Untwisting Paper\DCR4221\063014_lattices_Javier - UTP 1\Decon_reg\Decon_reg_236\Decon_reg_236_results\integrated_annotation\annotations.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\Original Untwisting Paper\DCR4221\063014_lattices_Javier - UTP 1\Decon_reg\Decon_reg_237\Decon_reg_237_results\straightened_annotations\straightened_annotations.csv
FI

FILE DNE: Y:\RyanC\Cell Tracking Project\Original Untwisting Paper\DCR4221\063014_lattices_Javier - UTP 1\Decon_reg\Decon_reg_256\Decon_reg_256_results\straightened_annotations\straightened_annotations.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\Original Untwisting Paper\DCR4221\063014_lattices_Javier - UTP 1\Decon_reg\Decon_reg_256\Decon_reg_256_results\integrated_annotation\annotations.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\Original Untwisting Paper\DCR4221\063014_lattices_Javier - UTP 1\Decon_reg\Decon_reg_257\Decon_reg_257_results\straightened_annotations\straightened_annotations.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\Original Untwisting Paper\DCR4221\063014_lattices_Javier - UTP 1\Decon_reg\Decon_reg_257\Decon_reg_257_results\integrated_annotation\annotations.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\Original Untwisting Paper\DCR4221\063014_lattices_Javier - UTP 1\Decon_reg\Decon_reg_258\Decon_reg_258_results\straightened_annotations\straightened_annotations.csv
FI

FILE DNE: Y:\RyanC\Cell Tracking Project\Original Untwisting Paper\DCR4221\063014_lattices_Javier - UTP 1\Decon_reg\Decon_reg_277\Decon_reg_277_results\straightened_annotations\straightened_annotations.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\Original Untwisting Paper\DCR4221\063014_lattices_Javier - UTP 1\Decon_reg\Decon_reg_277\Decon_reg_277_results\integrated_annotation\annotations.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\Original Untwisting Paper\DCR4221\063014_lattices_Javier - UTP 1\Decon_reg\Decon_reg_278\Decon_reg_278_results\straightened_annotations\straightened_annotations.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\Original Untwisting Paper\DCR4221\063014_lattices_Javier - UTP 1\Decon_reg\Decon_reg_278\Decon_reg_278_results\integrated_annotation\annotations.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\Original Untwisting Paper\DCR4221\063014_lattices_Javier - UTP 1\Decon_reg\Decon_reg_279\Decon_reg_279_results\straightened_annotations\straightened_annotations.csv
FI

FILE DNE: Y:\RyanC\Cell Tracking Project\Original Untwisting Paper\DCR4221\LE311_US_5min_2 - UTP 3\Decon_reg\RegB\Decon_reg_81\Decon_reg_81_results\straightened_seamcells\straightened_seamcells.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\Original Untwisting Paper\DCR4221\LE311_US_5min_2 - UTP 3\Decon_reg\RegB\Decon_reg_81\Decon_reg_81_results\straightened_annotations\straightened_annotations.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\Original Untwisting Paper\DCR4221\LE311_US_5min_2 - UTP 3\Decon_reg\RegB\Decon_reg_81\Decon_reg_81_results\straightened_lattice\straightened_lattice.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\Original Untwisting Paper\DCR4221\LE311_US_5min_2 - UTP 3\Decon_reg\RegB\Decon_reg_81\Decon_reg_81_results\seam_cell_final\seam_cells.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\Original Untwisting Paper\DCR4221\LE311_US_5min_2 - UTP 3\Decon_reg\RegB\Decon_reg_81\Decon_reg_81_results\integrated_annotation\annotations.csv
FILE DNE: Y:\RyanC\Cell Tracking Project\

In [7]:
# STEP 2: Check for outliers, uses parameters defined in config.json

window_size = config['settings']['outlier_removal']['window_size']
n_stdev = config['settings']['outlier_removal']['n_stdev']
compiled_data_no_outliers = copy.deepcopy(compiled_data)
for strain in compiled_data.keys():
    for pos in compiled_data[strain].keys():

        # determine outliers for both seam cells and annotations
        for cells_type in ['seam_cells', 'annotations']:
            for cell in compiled_data[strain][pos][cells_type].keys():
                coordinates = np.array(compiled_data[strain][pos][cells_type][cell]['coordinates'])

                # separate by x,y,z (index 0-2) and filter
                outlier_idxs = {} # set automatically removes repeats
                for dim_idx in range(3):
                    coord_dim_data = copy.deepcopy(coordinates[:, dim_idx])
                    data_filtered, outlier_idx, outlier_val = hampel_filter(coord_dim_data, 
                            n_sigmas=n_stdev, window=window_size)
                    coordinates[:, dim_idx] = data_filtered

                # replace time series outliers using median
                compiled_data_no_outliers[strain][pos][cells_type][cell]['coordinates'] = coordinates.tolist()

print('Step 2 Completed.')
# save information as intermediate step in workspace
compiled_json = json.dumps(compiled_data_no_outliers, sort_keys=True, indent=4)
compiled_json_filepath = os.path.join(
    workspace_folderpath, '2_compiled_data_no_outliers.json')
with open(compiled_json_filepath, "w") as f: 
    f.write(compiled_json)
print('Step 2 Data Logged.')

Step 2 Completed.
Step 2 Data Logged.


In [8]:
# STEP 3: Interpolate each to appropriate time scale
compiled_data_interp = copy.deepcopy(compiled_data_no_outliers)
seam_cells_on = config['settings']['interpolation']['seam_cells_on'] # in minutes
total_len = config['settings']['interpolation']['total_min'] # in minutes
interp_method = config['settings']['interpolation']['method']
min_timepoints_required = config['settings']['interpolation']['min_timepoints_required']
new_timepoints = np.linspace(0, total_len)

for strain in compiled_data_no_outliers.keys():
    for pos in compiled_data_no_outliers[strain].keys():
        
        cell_key = compiled_data_no_outliers[strain][pos]['cell_key']
        # interpolate for both seam cells and annotations
        for cells_type in ['seam_cells', 'annotations']:
            for cell in compiled_data_no_outliers[strain][pos][cells_type].keys():
                timepoints = np.array(compiled_data_no_outliers[strain][pos][cells_type][cell]['timepoints'])
                coordinates = np.array(compiled_data_no_outliers[strain][pos][cells_type][cell]['coordinates'])
                
                # CHECKS if there are a sufficient number of timepoints. if not, raise error.
                if timepoints.size < min_timepoints_required:
                    error_msg = "DATA ERROR: Insufficient timepoints in {}/{} for cell {} where only {} (volumes {}), less than the required {}, exist.".format(
                        strain, pos, cell, str(len(timepoints)), 
                        ",".join(timepoints.astype(str).tolist()), str(min_timepoints_required))
                    errors.append(create_error_dict(
                        strain=strain,
                        pos=pos,
                        cell_names=[cell],
                        volumes=timepoints,
                        error_type="Insufficient data to interpolate ({}<{})".format(
                            len(timepoints), min_timepoints_required),
                    ))
                    print(error_msg)
                    continue

                # handle seam cells and annotations differently because 
                # some seam cells appear after twitching begins
                vol_len = cell_key['end'] - cell_key['start']
                if cells_type == 'seam_cells':
                    # determine if strain starts after designated seam cell on
                    on_time = seam_cells_on[cell]
                elif cells_type == 'annotations':
                    on_time = 0
                    
                # every thing in interpolated timescale
                target_starting_idx = int(round(on_time * total_len))
                target_ending_idx = total_len
                starting_idx =  int(math.floor((min(timepoints) - cell_key['start'])/vol_len * total_len))
                ending_idx = int(math.ceil((1-(cell_key['end'] - max(timepoints))/vol_len) * total_len))
                
                new_coordinates = np.zeros((total_len, 3))
                new_timepoints = np.arange(total_len)
                # interpolate one dimension at a time
                for dim_idx in range(3):
                    cell_timepoints = timepoints.copy()
                    coord_dim_data = coordinates[:, dim_idx].copy()

                    # rescale time points to range, 0 to 46, rather than 8 to 54
                    cell_sp_rescaled = cell_timepoints - min(cell_timepoints) # not necessarily interval of 1
                    cell_sp_rescaled = cell_sp_rescaled/max(cell_sp_rescaled) * (ending_idx - starting_idx)
                
                    interp = interpolate.interp1d(cell_sp_rescaled, coord_dim_data,
                                     kind=interp_method) # get interp as if from 0, but shift below
                    
                    # new time scale for specific length (might be part of total). 
                    cell_sp_timepoints = np.arange(ending_idx - starting_idx)
                    interped_coords = interp(cell_sp_timepoints)
                    
                    # put the appropriate coordinates in
                    new_coordinates[starting_idx:ending_idx, dim_idx] = interped_coords # shifted
                    
                    # get rid of coordinates according to start time
                    new_coordinates[:target_starting_idx, dim_idx] = 0
                
                # replace time series outliers using median
                compiled_data_interp[strain][pos][cells_type][cell]['coordinates'] = new_coordinates.tolist()
                compiled_data_interp[strain][pos][cells_type][cell]['timepoints'] = new_timepoints.tolist()
                
# save information as intermediate step in workspace
print('Step 3 Completed.')
compiled_json = json.dumps(compiled_data_interp, sort_keys=True, indent=4)
compiled_json_filepath = os.path.join(
    workspace_folderpath, '3_compiled_data_interpolation.json')
with open(compiled_json_filepath, "w") as f: 
    f.write(compiled_json)
print('Step 3 Data Logged.')

Step 3 Completed.
Step 3 Data Logged.


In [9]:
# STEP 4: Generate seam cell warping model
exclude_seam_cells = ['QL', 'QR']
seam_cell_strains = config['data']['seam_cells']

z_padding = config['settings']['warping']['z_padding']
smoothing_method = config['settings']['smoothing']['method']
smoothing_window = config['settings']['smoothing']['window_size']

# check if it's a pre-generated warping model
pre_generated_model = False
if '.json' in seam_cell_strains[0]:
    pre_generated_model = True
    with open(seam_cell_strains[0]) as f:
        output_json = json.load(f)

    print("Using pre-generated warping model: {}".format(seam_cell_strains[0]))
else: # generate new warping model based on the strains given
    # go through files and look for the strain names
    warp_strain_names = []
    for seam_strain_folder in seam_cell_strains:
        cell_key_filepath_json = os.path.join(seam_strain_folder, 'cell_key.json')

        # if the json exists, use it
        if os.path.isfile(cell_key_filepath_json):
            with open(cell_key_filepath_json) as f:
                strain_cell_key = json.load(f)
                warp_strain_names.append(strain_cell_key['name'])

    # if not warp strain names, use a cached warping model
    seam_warp_model_folderpath = config['data']['seam_cells']
    print("Using the following strains for warping: {}".format(", ".join(warp_strain_names)))

    # first combine all the necessary data for seam cells
    warp_model_by_cell = {}
    warp_model_by_cell_unsmoothed = {} # for testing/debugging. not used lated
    for strain in compiled_data_interp.keys():
        for pos in compiled_data_interp[strain].keys():

            if pos not in warp_strain_names:
                continue

            for cell in compiled_data_interp[strain][pos]['seam_cells'].keys():
                if cell in exclude_seam_cells:
                    continue 

                if cell not in warp_model_by_cell.keys():
                    warp_model_by_cell[cell] = []

                seam_cell_coordindates = np.array(
                    compiled_data_interp[strain][pos]['seam_cells'][cell]['coordinates'])
                warp_model_by_cell[cell].append(
                    seam_cell_coordindates.tolist())

    # average/smooth all coordinates by cell for warping model
    for cell in warp_model_by_cell.keys():
        # average all the coordinates, step 5
        cell_coords = np.array(warp_model_by_cell[cell])
        nan_mean = np.nanmean(np.where(cell_coords!=0, cell_coords, np.nan), axis=0) # nonzero mean
        nan_mean = np.nan_to_num(nan_mean) # replace the nan with 0
        warp_model_by_cell[cell] = nan_mean # np.average(warp_model_by_cell[cell], axis=0) # full coordinates
        
        warp_model_by_cell_unsmoothed[cell] = nan_mean.tolist() # testing/debugging. not used later.

        # smooth, step 7
        coordinates = warp_model_by_cell[cell]
        for dim_idx in range(3):
            coord_dim_data = coordinates[:, dim_idx].copy()
            data_smoothed = smoothing(coord_dim_data, smoothing_window, method=smoothing_method)
            coordinates[:, dim_idx] = data_smoothed
            
            # extra z padding so that anterior cells aren't compressed at beginning
            if dim_idx == 2:
                coordinates[:, dim_idx] += z_padding

        # convert to list
        warp_model_by_cell[cell] = coordinates.tolist() # full coordinates

    # convert to all cells for each timepoint
    total_len = config['settings']['interpolation']['total_min'] # in minutes
    new_timepoints = np.arange(0, total_len)
    seam_cells_on = config['settings']['interpolation']['seam_cells_on']
    sorted_seam_cells = sorted(warp_model_by_cell.keys()) # maintain some order
    warp_model_by_timepoint = []
    warp_model_by_timepoint_unsmoothed = [] # not used
    for timepoint in new_timepoints.astype(int):
        timepoint_coords_by_cell = []
        timepoint_coords_by_cell_unsmoothed = []
        for seam_cell in sorted_seam_cells:
            all_timepoint_coords = np.array(warp_model_by_cell[seam_cell])
            seam_cell_coords = all_timepoint_coords[timepoint, :].tolist()
            timepoint_coords_by_cell.append(seam_cell_coords)
            
            # non-smoothed. for debugging/not used later
            all_timepoint_coords_unsmoothed = np.array(warp_model_by_cell_unsmoothed[seam_cell])
            seam_cell_coords_unsmoothed = all_timepoint_coords_unsmoothed[timepoint, :].tolist()
            timepoint_coords_by_cell_unsmoothed.append(seam_cell_coords_unsmoothed)
                
        warp_model_by_timepoint.append(timepoint_coords_by_cell)
        warp_model_by_timepoint_unsmoothed.append(timepoint_coords_by_cell_unsmoothed) # not used

    # create output structure
    output_json = {}
    output_json['seam_cells'] = sorted_seam_cells
    output_json['coordinates'] = warp_model_by_timepoint
    
    # unsmoothed / DELETE LATER!
    output_json_unsmoothed = {} 
    output_json_unsmoothed ['seam_cells'] = sorted_seam_cells
    output_json_unsmoothed ['coordinates'] = warp_model_by_timepoint_unsmoothed 

print('Step 4 Completed.')
# save information as intermediate step in workspace
warping_model = output_json
compiled_json = json.dumps(output_json, sort_keys=True, indent=4)
compiled_json_filepath = os.path.join(
    workspace_folderpath, '4_seam_cell_warping_model.json')
with open(compiled_json_filepath, "w") as f: 
    f.write(compiled_json)
    
# save information as intermediate step in workspace / NOT USED LATER
warping_model_unsmoothed = output_json_unsmoothed
compiled_json_unsmoothed = json.dumps(output_json_unsmoothed, sort_keys=True, indent=4)
compiled_json_unsmoothed_filepath = os.path.join(
    workspace_folderpath, '4_seam_cell_warping_model_unsmoothed.json')
with open(compiled_json_unsmoothed_filepath, "w") as f: 
    f.write(compiled_json_unsmoothed)
    
print('Step 4 Data Logged.')

Using the following strains for warping: DCR4221_063014_lattices_Javier - UTP 1, DCR4221_2015_Pos2, DCR4221_2015_Pos3, DCR4221_2015_Pos4, DCR4221_2015_Pos5, OD1599_NU_1206_Pos2, OD1599_NU_1127_Pos3, OD1599_NU_1126_Pos0, ï»¿DCR6485NU_011419_Pos0, ï»¿DCR6485NU_011419_Pos4, ï»¿DCR6485NU_021020_Pos2, JCC596_NU_0911_Pos3, JCC596_NU_0911_Pos2, JCC596_NU_0826_Pos3, KP9305_Pos0, KP9305_Pos4, KP9305_Pos2, RW131_052918_Pos0, RW131_052918_Pos1, RW10752_0225_Pos0, RW10752_0225_Pos4, RW10752_0225_Pos6, RW10752_0312_Pos2, RW10752_0312_Pos1, RW10752_0225_Pos0, RW10584_NU_051817, RW10584_NU_052517, RW10584_NU_101717, RW10598_Pos4-bgsub_95_iteration_5, RW10598_598_Slitscan_6um_5min_Pos1, RW10598_Pos0
Step 4 Completed.
Step 4 Data Logged.


In [14]:
# STEP 5: Warp strains to warping model
# we need to do this first because seam cell information
y_correction = config['settings']['warping']['y_correction']
timepoints = np.arange(0, total_len).astype(int).tolist()
compiled_data_warped = copy.deepcopy(compiled_data_interp)
for strain in compiled_data_interp.keys():
    for pos in compiled_data_interp[strain].keys():
        print('Warping {}, Position {}'.format(strain, pos))
        for timepoint in timepoints:
            # get warp from/to model at timepoint (the seam cells)
            pos_seam_cells = compiled_data_interp[strain][pos]['seam_cells']
            warp_to_seam_cell_names = warping_model['seam_cells']
            warp_to_seam_cell_coords = warping_model['coordinates']
            warp_to_at_timepoint = warp_to_seam_cell_coords[timepoint] # contains all warping data
            
            # however, we might not need/have all the data so here we match what we have
            warp_to = []
            warp_from = []
            for warp_seam_cell_idx, warp_to_seam_cell_name in enumerate(warp_to_seam_cell_names):
                if warp_to_seam_cell_name in pos_seam_cells.keys():
                    # print(warp_to_seam_cell_name)
                    # add to warp to
                    warp_to_coords = warp_to_at_timepoint[warp_seam_cell_idx]
                    warp_to.append(warp_to_coords)
                    
                    # add to warp from
                    all_time_coords = pos_seam_cells[warp_to_seam_cell_name]['coordinates']
                    warp_from_timepoint_coord = all_time_coords[timepoint]
                    warp_from.append(warp_from_timepoint_coord)
            
            warp_to = np.array(warp_to)
            warp_from = np.array(warp_from)
            
            # warp each cell at timepoint, including the seam cells themselves
            for cell_type in ['seam_cells', 'annotations']:
                # get cell order for warping
                sorted_cell_names = compiled_data_interp[strain][pos][cell_type].keys()
                ordered_coord_list = []
                for cell_name in sorted_cell_names:
                    old_coords = [compiled_data_interp[strain][pos][cell_type][cell_name]['coordinates'][timepoint]]
                    old_coords = np.array(old_coords).flatten().tolist() # ensure correct data shape
                    ordered_coord_list.append(old_coords)
                ordered_coord_list = np.array(ordered_coord_list)
                
                # check if there are cells to warp
                if ordered_coord_list.size == 0:
                    continue
                
                # warping
                new_coords = thin_plate_spline_warp_c_elegans(warp_from, warp_to, ordered_coord_list)
                if y_correction:
                    new_coords = y_warp_correction(warp_from, warp_to, ordered_coord_list, new_coords)
                new_coords = new_coords.tolist()
                
                # assigning
                for cell_idx, cell_name in enumerate(sorted_cell_names):
                    # if the old coordinates used to be 0 (ignored), then keep it that way
                    old_coords = compiled_data_interp[strain][pos][cell_type][cell_name]['coordinates'][timepoint]
                    
                    if old_coords == [0, 0, 0]:
                        assign_coords = old_coords
                    else:
                        assign_coords = new_coords[cell_idx]
                        
                    compiled_data_warped[strain][pos][cell_type][cell_name]['coordinates'][timepoint] = \
                        assign_coords
    
print('Step 5 Completed.')
# save information as intermediate step in workspace
output_json = compiled_data_warped
compiled_json = json.dumps(output_json, sort_keys=True, indent=4)
compiled_json_filepath = os.path.join(
    workspace_folderpath, '5_compiled_data_warped.json')
with open(compiled_json_filepath, "w") as f: 
    f.write(compiled_json)
print('Step 5 Data Logged.')

Warping OD1599_NU, Position OD1599_NU_1206_Pos2
Warping OD1599_NU, Position OD1599_NU_1127_Pos3
Warping OD1599_NU, Position OD1599_NU_1126_Pos0
Warping DCR6485_RPM1_NU, Position ï»¿DCR6485NU_011419_Pos0
Warping DCR6485_RPM1_NU, Position ï»¿DCR6485NU_011419_Pos4
Warping DCR6485_RPM1_NU, Position ï»¿DCR6485NU_021020_Pos2
Warping JCC596_NU, Position JCC596_NU_0911_Pos3
Warping JCC596_NU, Position JCC596_NU_0911_Pos2
Warping JCC596_NU, Position JCC596_NU_0826_Pos3
Warping KP9305_NU, Position KP9305_Pos0
Warping KP9305_NU, Position KP9305_Pos4
Warping KP9305_NU, Position KP9305_Pos2
Warping RW10131, Position RW131_052918_Pos0
Warping RW10131, Position RW131_052918_Pos1
Warping RW10896, Position RW10752_0225_Pos0
Warping RW10896, Position RW10752_0225_Pos4
Warping RW10896, Position RW10752_0225_Pos6
Warping RW10752_NU, Position RW10752_0312_Pos2
Warping RW10752_NU, Position RW10752_0312_Pos1
Warping RW10752_NU, Position RW10752_0225_Pos0
Warping RW10584, Position RW10584_NU_051817
Warping RW

In [15]:
# STEP 6: Reformat data to average points/calculate statistics
step_data = copy.deepcopy(compiled_data_warped)
data_by_cell = {'seam_cells':{}, 'annotations':{}}
for strain in step_data.keys():
    for pos in step_data[strain].keys():
        for cell_type in ['seam_cells', 'annotations']:
            for cell in step_data[strain][pos][cell_type].keys():
                if cell not in data_by_cell[cell_type].keys():
                    data_by_cell[cell_type][cell] = []

                cell_coordindates = np.array(
                    step_data[strain][pos][cell_type][cell]['coordinates'])
                data_by_cell[cell_type][cell].append(
                    cell_coordindates.tolist()) # should be putting full coordinates at end

# perform operations/statistics on all coordinates by cell
data_by_cell_stats = {'mean': {}, 'std_dev': {}, 'sem': {}, 'ci_95_lower': {}, 'ci_95_upper': {}}
for calc_type in data_by_cell_stats.keys():
    data_by_cell_stats[calc_type] = copy.deepcopy(data_by_cell)
    
    for cell_type in ['seam_cells', 'annotations']:
        # seam cells should just result in the average warping model
        for cell in data_by_cell_stats[calc_type][cell_type].keys():
            cell_coords = np.array(data_by_cell_stats[calc_type][cell_type][cell])

            if calc_type == 'mean': # average all the coordinates
                nan_mean = np.nanmean(np.where(cell_coords!=0, cell_coords, np.nan), axis=0) # nonzero mean
                nan_mean = np.nan_to_num(nan_mean) # replace the nan with 0
                data_by_cell_stats[calc_type][cell_type][cell] = nan_mean.tolist()
            elif calc_type == 'std_dev': # find standard deviation of all coordinates
                data_by_cell_stats[calc_type][cell_type][cell] = np.std(cell_coords, axis=0).tolist()
            elif calc_type == 'sem': # find standard error of all coordinates
                data_by_cell_stats[calc_type][cell_type][cell] = st.sem(cell_coords, axis=0).tolist()
            elif 'ci_95' in calc_type: # find 95% confidence interval
                ci_ppf = st.t.ppf((1+0.95)/2, cell_coords.shape[0])
                time_dim_mean = np.array(data_by_cell_stats['mean'][cell_type][cell])
                time_dim_sem = np.array(data_by_cell_stats['sem'][cell_type][cell])
                
                if 'lower' in calc_type:
                    ci_val = time_dim_mean - ci_ppf*time_dim_sem
                elif 'upper' in calc_type:
                    ci_val = time_dim_mean + ci_ppf*time_dim_sem
                
                data_by_cell_stats[calc_type][cell_type][cell] = ci_val.tolist()
                        
    print('{} Calculation Completed.'.format(calc_type.capitalize()))

print('Step 6 Completed.')
# save information as intermediate step in workspace
output_json = data_by_cell_stats
compiled_json = json.dumps(output_json, sort_keys=True, indent=4)
compiled_json_filepath = os.path.join(
    workspace_folderpath, '6_cell_coord_stats_by_timepoint.json')
with open(compiled_json_filepath, "w") as f: 
    f.write(compiled_json)
print('Step 6 Data Logged.')

C:\Users\xusz\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: Mean of empty slice


Mean Calculation Completed.
Std_dev Calculation Completed.
Sem Calculation Completed.
Ci_95_lower Calculation Completed.
Ci_95_upper Calculation Completed.
Step 6 Completed.
Step 6 Data Logged.


In [16]:
# STEP 7: Perform spatial moving average for each cell
data_stats = copy.deepcopy(data_by_cell_stats)
warping_cells = config['settings']['warping']['seam_cells']
smoothing_method = config['settings']['smoothing']['method']
smoothing_window = config['settings']['smoothing']['window_size']

# average all coordinates by cell and for all stats
for calc_type in data_stats.keys():
    for cell_type in ['seam_cells', 'annotations']: # don't re-smooth the seam cells
        # seam cells should just result in the average warping model
        for cell in data_stats[calc_type][cell_type].keys():
            # Will smooth Q cells, data_stats will have the original for other cells
            if cell in warping_cells:
                continue
                
            # spatial average all the coordinates by time
            coordinates = np.array(data_stats[calc_type][cell_type][cell])
            if not np.isnan(coordinates).all(): # sometimes it will be all nan
                # go through each axis
                for dim_idx in range(3):
                    coord_dim_data = coordinates[:, dim_idx].copy()
                    data_smoothed = smoothing(coord_dim_data, smoothing_window, method=smoothing_method)
                    coordinates[:, dim_idx] = data_smoothed
                    
            data_stats[calc_type][cell_type][cell] = coordinates.tolist()

print('Step 7 Completed.')
# save information as intermediate step in workspace
output_json = data_stats
compiled_json = json.dumps(output_json, sort_keys=True, indent=4)
compiled_json_filepath = os.path.join(
    workspace_folderpath, '7_cell_coord_stats_by_timepoint_smoothed.json')
with open(compiled_json_filepath, "w") as f: 
    f.write(compiled_json)
print('Step 7 Data Logged.')

Step 7 Completed.
Step 7 Data Logged.


In [17]:
# STEP 8: Convert to csv in MIPAV format + generate error file
scale = config['settings']['voxel_to_micron_scale']
labels_on = config['settings']['mipav_output']['labels_on']
# create timepoints array again just to be safe
total_len = config['settings']['interpolation']['total_min'] # in minutes
cell_info_file = config['settings']['mipav_output']['cell_info']
cell_info = None
with open(cell_info_file) as f:
    cell_info = json.load(f)
timepoints = np.arange(0, total_len)

# generate place to put all the files
output_folder = os.path.join(workspace_folderpath, 'output')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
                

for cell_type in ['seam_cells', 'annotations']:
    cells = data_stats['mean'][cell_type] # only output the mean

    for cell_name in cells.keys():
        # causes file issues + handle escape sequences
        file_cell_name = cell_name.replace('/','_').replace(' ', '_')
        filename_csv = file_cell_name +'.csv'
        filepath_csv = os.path.join(output_folder, filename_csv)

        coordinates = data_stats['mean'][cell_type][cell_name]
        new_table = pd.DataFrame(coordinates) * scale[0]
        new_table.columns = ['x', 'y', 'z']
        new_table.insert(0, "timepoints", timepoints, True)
        # colors
        if cell_name.lower() in cell_info.keys():
            color_timepoints = list(cell_info[cell_name.lower()]['colors'].keys())
            colors = [cell_info[cell_name.lower()]['colors'][color_timepoint] for color_timepoint in color_timepoints]
            colors = np.array(colors)
            color_timepoints = np.array(color_timepoints).astype(float) # because originally string
            
            # color_timepoints_sorted_idx, color_timepoints_sorted = (np.argsort(color_timepoints), np.sort(color_timepoints))
            # colors_sorted = np.array(colors[color_timepoints_sorted_idx]) # get sorted order
            
            # if there's one, just use nearest, otherwise use linear
            if color_timepoints.shape[0] == 1: # use single value
                placeholder_color = np.ones((total_len,))

                for dim_idx, channel in enumerate(['R', 'G', 'B']):
                    # color calculation/interpolation goes here
                    dim_color = colors[0, dim_idx]
                    interped_colors = placeholder_color * dim_color

                    new_table.insert(dim_idx + 4, channel, interped_colors, True)
                
            else:
                color_interp_method = 'linear'
                for dim_idx, channel in enumerate(['R', 'G', 'B']):
                    # color calculation/interpolation goes here
                    interp = interpolate.interp1d(color_timepoints, colors[:, dim_idx], kind=color_interp_method)
                    interped_colors = np.round(interp(timepoints))

                    new_table.insert(dim_idx + 5, channel, interped_colors, True)
                
        else: # set color to white
            placeholder_color = np.ones((total_len,)) * 255
            new_table.insert(4, "R", placeholder_color, True)
            new_table.insert(5, "G", placeholder_color, True)
            new_table.insert(6, "B", placeholder_color, True)
            
        if labels_on:
            new_table.insert(7, "label", np.ones((total_len,)), True)
        else:
            new_table.insert(7, "label", np.zeros((total_len,)), True)
            
        # format each column
        for coord_col in ['x', 'y', 'z']: # as scientific notation
            new_table[coord_col] = new_table[coord_col].map(lambda x: '%.6e' % x)
            
        for color_col in ['R', 'G', 'B', 'label']: # as integers
            new_table[color_col] = new_table[color_col].map(lambda x: '%d' % x)

        new_table.to_csv(filepath_csv, index=False, header=False)
print('CSVs generated.')
create_error_xlsx(workspace_folderpath, errors)
print('Errors Logged.')
full_output_folderpath = os.path.join(os.getcwd(), output_folder)
print('Step 8 Completed.')
print('Output location: {}'.format(full_output_folderpath))

CSVs generated.
Errors Logged.
Step 8 Completed.
Output location: Y:\RyanC\model_building_code\workspace\2021_01_08-Full_Model_ywarptest\output
